In [27]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import pandas_profiling as pp
import seaborn as sns
import warnings
import os

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)



In [28]:
# Leitura do dataset refinado
diretorio_acidentes = 'data/refined/acidentes_recife.csv'
df_acidentes = pd.read_csv(diretorio_acidentes, sep = ',')
       

In [29]:
df_acidentes.head(3)

,Unnamed: 0,data,natureza_acidente,situacao,bairro,endereco,complemento,bairro_cruzamento,tipo,descricao,Protocolo,natureza,auto_bin,moto_bin,ciclom_bin,ciclista_bin,pedestre_bin,onibus_bin,caminhao_bin,viatura_bin,outros_bin,vitimas_bin,vitimasfatais_bin,dia_da_semana,mes,ano
0,0,2021-01-01,Não fatal,FINALIZADA,MADALENA,RUA BENFICA,EM FRENTE AO EXTRA NA SAIDA,MADALENA,COLISÃO FRONTAL,NaN,NaN,NaN,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Friday,1.0,2021.0
1,1,2021-01-01,Não fatal,FINALIZADA,VÁRZEA,AV CAXANGA,EM FRENTE A UPA DA CAXANGA,VÁRZEA,COLISÃO TRASEIRA,NaN,NaN,NaN,0.25,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Friday,1.0,2021.0
2,2,2021-01-02,Não fatal,FINALIZADA,VASCO DA GAMA,RUA RESPLENDOR,AO LADO DO COLEGIO ADERBAL GALVÃO EM FRE...,VASCO DA GAMA,COLISÃO COM CICLISTA,NaN,NaN,NaN,0.00,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Saturday,1.0,2021.0


# Estatísticas descritivas

Para explorar e entender nossos dados de forma eficiente, vamos utilizar o pandas-profiling. Essa biblioteca nos permite gerar análises descritivas de maneira automatizada e simplificada. Dessa forma, aproximamos nosso trabalho de projetos reais de ciência de dados.

In [30]:
df_acidentes.describe()

,Unnamed: 0,descricao,auto_bin,moto_bin,ciclom_bin,ciclista_bin,pedestre_bin,onibus_bin,caminhao_bin,viatura_bin,outros_bin,vitimas_bin,vitimasfatais_bin,mes,ano
count,36612.000000,0.0,36612.000000,36612.000000,36612.000000,36612.000000,36612.000000,36612.000000,36612.000000,36612.000000,36612.000000,36612.000000,36612.000000,36444.000000,36444.000000
mean,18309.427838,NaN,0.008522,0.003455,0.001202,0.003127,0.004391,0.008713,0.000068,0.000027,0.000048,0.000109,0.000055,6.744265,2022.797498
std,10569.241058,NaN,0.047352,0.029938,0.034647,0.029048,0.036670,0.067684,0.006128,0.005226,0.006533,0.008467,0.006662,3.440386,0.542352
min,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2021.000000
25%,9156.750000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2023.000000
50%,18309.500000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000,2023.000000
75%,27462.250000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,2023.000000
max,36615.000000,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,12.000000,2024.000000


In [31]:
report = pp.ProfileReport(df_acidentes)
report.to_file('Análise Exploratória de Dados.html')

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 90.88it/s]


OBS: Para visualização correta, é necessário baixar o html gerado e abrir no navegador.